In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, utils

import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from sklearn import preprocessing
from PIL import Image
import time
import os
import copy
from collections import Counter

In [2]:
x1 = torch.rand((16, 100))
x2 = torch.rand((16, 50))

m1 = nn.Linear(100, 200)
m2 = nn.Linear(250, 1)

z = m1(x1)
z = torch.cat([z, x2], dim=1)
out = m2(z)

In [3]:
def print_hook(self, ins):
    print(ins[0].shape)
    # print(out.shape)

In [4]:
m1 = nn.Linear(100, 200)
h = m1.register_forward_pre_hook(print_hook)
m2 = nn.Linear(250, 1)

In [5]:
z = m1(x1)
z = torch.cat([z, x2], dim=1)
out = m2(z)

torch.Size([16, 100])


In [6]:
h.remove()
z = m1(x1)
z = torch.cat([z, x2], dim=1)
out = m2(z)

In [173]:
fc1 = nn.Linear(100, 200)
fc2 = nn.Linear(250, 1)

In [181]:
class m(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = copy.deepcopy(fc1)
        self.fc2 = copy.deepcopy(fc2)
        def cat_hook(sef, ins):
            return tuple([torch.cat([ins[0], x2], dim=1)])
        self.fc2.register_forward_pre_hook(cat_hook)
        
    def forward(self, x1, x2):
        out = self.fc2(self.fc1(x1))
        return out
m1 = m()

In [182]:
y1 = m1(x1, x2)

In [183]:
%timeit m1(x1, x2).shape

51.9 µs ± 484 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [184]:
class mm(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = copy.deepcopy(fc1)
        self.fc2 = copy.deepcopy(fc2)

    def forward(self, x1, x2):
        z = self.fc1(x1)
        z = torch.cat([z, x2], dim=1)
        out = self.fc2(z)
        return out
m2 = mm()

In [185]:
y2 = m2(x1, x2)

In [186]:
%timeit m2(x1, x2).shape

59.1 µs ± 14.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [189]:
for i in range(10):
    x1 = torch.rand((16, 100))
    x2 = torch.rand((16, 50))
    print((m1(x1, x2) == m2(x1, x2)).all())

tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)


In [180]:
(y1 == y2).all()

tensor(True)

In [14]:
class PriceModel(nn.Module):
    '''Neural Network for price prediction.'''

    def __init__(self, num_ftrs=0, hidden_units=None, fine_tune=2):
        '''
            num_ftrs (int): Number of features added into this model.
            hidden_units (list): Number of neurons used in each fully connected layers.
            fine_tune (int): Number of CNN blocks (counting backwards) to fine tune.

        '''
        super().__init__()
        assert type(fine_tune) == int and fine_tune < 5, "fine_tune should be a non-negative int smaller than 5"

        cnn = models.resnet152(pretrained=True)
        for param in cnn.parameters():
            param.requires_grad = False
        if fine_tune > 0:
            for param in cnn.layer4.parameters():
                param.requires_grad = True
        if fine_tune > 1:
            for param in cnn.layer3.parameters():
                param.requires_grad = True
        if fine_tune > 2:
            for param in cnn.layer2.parameters():
                param.requires_grad = True
        if fine_tune > 3:
            for param in cnn.layer1.parameters():
                param.requires_grad = True
        cnn.fc = nn.Identity()

        fc_layers = nn.Sequential()
        if not hidden_units:
            fc_layers.add_module("fc1", nn.Linear(2048+num_ftrs, 1))
        else:
            linear_units = [2048+num_ftrs] + hidden_units
            for ii, (ins, outs) in enumerate(zip(linear_units[:-1], linear_units[1:])):
                fc_layers.add_module("fc"+str(ii+1), nn.Linear(ins, outs))
                fc_layers.add_module("activ"+str(ii+1), nn.Sigmoid())
            fc_layers.add_module("fc"+str(ii+2), nn.Linear(outs, 1))

        self.cnn = cnn
        self.fc_layers = fc_layers
        self.num_ftrs = num_ftrs
        self.fint_tune = fine_tune
        self.hidden_units = hidden_units

    def forward(self, images, ftrs):
        if self.training:
            z = self.cnn(images)
            z = torch.cat([z, ftrs], dim=1)
            out = self.fc_layers(z)
            return out

In [15]:
test = PriceCNN(num_ftrs=5, fine_tune=2, hidden_units=[32])

In [12]:
x1 = torch.rand((16, 3, 224, 224))
x2 = torch.rand(16, 5)

In [13]:
test(x1, x2).shape

torch.Size([16, 1])

In [2]:
temp = models.resnet152(pretrained=True)

In [3]:
[child[0] for child in temp.named_children()]

['conv1',
 'bn1',
 'relu',
 'maxpool',
 'layer1',
 'layer2',
 'layer3',
 'layer4',
 'avgpool',
 'fc']

In [4]:
x = torch.rand((16, 3, 224, 224))

In [6]:
z = temp.conv1(x)
z = temp.bn1(z)
z = temp.relu(z)
z = temp.maxpool(z)
z = temp.layer1(z)
z = temp.layer2(z)
z = temp.layer3(z)
z = temp.layer4(z)
print(z.shape)
z = temp.avgpool(z)
print(z.shape)
z = temp.fc(z.squeeze())

torch.Size([16, 2048, 7, 7])
torch.Size([16, 2048, 1, 1])


In [ ]:
z.shape

In [20]:
y = temp.avgpool(z)

In [21]:
y.shape

torch.Size([16, 2048, 1, 1])

In [25]:
temp.avgpool

AdaptiveAvgPool2d(output_size=(1, 1))

In [36]:
print("{:^5} 123".format("val"))

 val  123


In [51]:
print('{:^5} Loss: {:.4f}   MAE: {:.2f}   MAEP: {:.2f} %'.format("train".upper(), 1.2, 12.6, 100*0.123647))

TRAIN Loss: 1.2000   MAE: 12.60   MAEP: 12.36 %


In [2]:
torch.load("../PriceModel.pt")

ModuleNotFoundError: No module named 'utils'